In [1]:
import pandas as pd
import numpy as np
import os
import json
import re
import random
from scipy.sparse import coo_matrix
from scipy.io import mmread, mmwrite, mminfo
import dgl
import meta_2rel_v2, meta_3rel_v2
# 用 base 环境，anaconda3 环境的 pandas 版本不对
pd.__version__

'0.25.1'

两种方法统计四种关系的数量：一个是统计四种关系 mtx 中的 nnz，一种是统计csv表格里的行数，两种方法算出来的结果不一样，最终还是用关系csv表格里的行数

In [31]:
def path_stats(PATH_NAME):
    N_by_year = []
    for year in range(2003,2022):
        path = mmread(f'mat_by_year/{year}/{PATH_NAME}.mtx')
        N_by_year.append(path.nnz)
    return N_by_year, np.sum(N_by_year)

n_gaoguan_by_year, n_gaoguan = path_stats('gaoguan')
_, n_trans = path_stats('guanlian')
_, n_gud = path_stats('gud_&_sub')
n_qin_by_year, n_qinshu = path_stats('qinshu')
print(f'每种关系数量：高管: {n_gaoguan}, 亲属: {n_qinshu}, 股东及子公司: {n_gud}, 关联交易: {n_trans}, 总计: {n_gaoguan+n_qinshu+n_gud+n_trans}')

每种关系数量：高管: 844511, 亲属: 33232, 股东及子公司: 1371404, 关联交易: 1113171, 总计: 3362318


In [33]:
# 五种关系
subfirm = pd.read_csv('../上市公司子公司情况表/subsidiary_fullname_matched.csv',encoding='gb18030')
qinshu = pd.read_csv('../股权及亲属关系/qinshu_no_dup_person.csv',encoding='gb18030')
gaoguan = pd.read_csv('../高管关系/gaoguan_no_dup_person.csv',encoding='gb18030')
guanlian = pd.read_csv('../关联方交易/guanlian_no_dup_person.csv',encoding='gb18030')
gudong = pd.read_csv('../十大股东文件/gudong_not_fin.csv',encoding='gb18030')
# gudong = pd.read_csv('../十大流通股股东/gudong_no_dup_person.csv',encoding='gb18030')

#读取train_test数据集
# train_test = pd.read_csv('../train_test_x10.csv',encoding='gb18030')
stk_fullname = pd.read_csv('../stk_fullname.csv',encoding='gb18030')
fullname_id_dict = dict((stk_fullname.FullName[i],stk_fullname.Stkcd[i]) for i in stk_fullname.index)

print('每种关系的数量：亲属:%d; 高管:%d; 关联交易:%d; 股东关系(包括子公司):%d; 共计%d' % 
      (len(qinshu), len(gaoguan), len(guanlian), len(gudong)+len(subfirm), len(subfirm)+len(qinshu)+len(gaoguan)+len(guanlian)+len(gudong)) )

每种关系的数量：亲属:16684; 高管:844626; 关联交易:686116; 股东关系(包括子公司):1412443; 共计2959869


### 生成上市公司、非上市公司、自然人三个列表

完整的上市公司列表来源于 code_for_CSMAR.xlsx，code_for_CSMAR 是由 会计信息质量-财务指标/all_corp_fin_data.xlsx 里面的上市公司提取出来的，就是财务指标完整的上市公司

In [16]:
n_listed_corp = 0
n_unlisted_corp = 0
n_people = 0

# 先生成所有上市公司的一个列表
code_for_CSMAR = pd.read_excel('../code_for_CSMAR.xlsx',encoding = 'utf-8')
listed_corp, unlisted_corp = set(code_for_CSMAR.code), set()
# 子公司里面有一些是上市公司
# unlisted_corp = set(subfirm.subsidiary)-set([str(i).zfill(6) for i in listed_corp])
for i in set(subfirm.subsidiary):
    try:
        # 把子公司里面的上市公司代码加到listed_corp集合中
        listed_corp.add(int(i))
    except:
        unlisted_corp.add(i)

# 高管关系中的manager和亲属关系中的person1和person2都是个人
people = set(list(gaoguan.manager)+list(qinshu.person1)+list(qinshu.person2))
print('子公司中有%d家上市公司,剔除,剩下的作为初始的非上市公司集合' % len(set(subfirm.subsidiary)-unlisted_corp))

子公司中有134家上市公司,剔除,剩下的作为初始的非上市公司集合


In [17]:
# 股东关系,holded不可能是个人，holder可能是个人、上市公司、非上市公司
last_name = r'^[赵钱孙李周吴郑王冯陈褚卫蒋沈韩杨朱秦尤许何吕施张孔曹严华金魏陶姜戚谢邹喻柏水窦章云苏潘葛奚范彭郎鲁韦昌马苗凤花方俞任袁柳酆鲍史唐费廉岑薛雷贺倪汤滕殷罗毕郝邬安常乐于时傅皮卞齐康伍余元卜顾孟平黄和穆萧尹姚邵湛汪祁毛禹狄米贝明臧计伏成戴谈宋茅庞熊纪舒屈项祝董梁杜阮蓝闵席季麻强贾路娄危江童颜郭梅盛林刁钟徐邱骆高夏蔡田樊胡凌霍虞支柯昝管卢莫经房裘缪干解应宗丁宣贲邓郁单杭洪包诸左石崔吉钮龚程嵇邢滑裴陆荣翁荀羊於惠甄麴家封芮羿储靳汲邴糜松井段富巫乌焦巴弓牧隗山谷车侯宓蓬全郗班仰秋仲伊宫宁仇栾暴甘钭厉戎祖武符刘景詹束叶幸司韶郜黎蓟薄印宿白怀蒲邰从鄂索咸籍赖卓蔺屠蒙池乔阴郁胥能苍双闻莘党翟谭贡劳逄姬申扶堵冉宰郦雍舄璩桑桂濮牛寿通边扈燕冀郏浦尚农温别庄晏柴瞿阎充慕连茹习宦艾鱼容向古易慎戈廖庾终暨居衡步都耿满弘匡国文寇广禄阙东殴殳沃利蔚越夔隆师巩厍聂晁勾敖融冷訾辛阚那简饶空曾毋沙乜养鞠须丰巢关蒯相查後荆红游竺权逯盖益桓公晋楚闫法汝鄢涂钦岳帅缑亢况后有商牟佘佴伯赏南宫墨哈谯笪年爱阳佟言福欧候劳原卿]'
firm_suffix = r'公司|有限|股份|企业|责任|集团|银行|证券|基金|保险|科技|资管|Ltd|LTD|INC|Inc|LIMITED|A/C|中国|Bank|平台|Corp|CORP|医院|欧洲|控股|大学|煤炭|地产|绿化|服饰|小学|船厂|发展|药业|城投|建投|实业|增长'
for i in gudong.index:
    holded = gudong.loc[i,'Stkcd']
    holder = gudong.loc[i,'shareholder']
    # 后半部分的数字股票代码是字符串，用type = int识别不出来
    try: # 判断是否是数字或者数字字符串
        holded = int(holded)
        listed_corp.add(holded)
    except:
        unlisted_corp.add(holded)
    
    try: # 判断是否是数字或者数字字符串
        holder = int(holder)
        listed_corp.add(holder)
    except:
        #if type(holder)==int or re.match('^[0-9]+',holder) is not None:
        if holder in unlisted_corp or holder in people:
            continue
        elif len(re.findall(firm_suffix,holder)) > 0: # 不要用re.match，有些识别不出来
            unlisted_corp.add(holder)
        elif len(re.findall(r'^[^0-9A-Za-z]+-[0-9]+',holder)) > 0:
            people.add(holder)
        elif len(holder)<=4 and len(re.findall(last_name,holder)) > 0:
            people.add(holder)
        else:
            unlisted_corp.add(holder)

In [18]:
# 关联关系
for i in guanlian.index:
    seller = guanlian.loc[i,'seller']
    buyer = guanlian.loc[i,'buyer']
    try:
        seller = int(seller)
        listed_corp.add(seller)
    except:
        if seller not in unlisted_corp and seller not in people:
            if len(re.findall(firm_suffix,seller)) > 0 :
                unlisted_corp.add(seller)
            elif len(seller)<=4 and len(re.findall(last_name,seller)) > 0:
                people.add(seller)
            elif len(re.findall(r'^[^0-9A-Za-z]+-[0-9]+',seller)) > 0:
                people.add(seller)
            else:
                unlisted_corp.add(seller)

    try:
        buyer = int(buyer)
        listed_corp.add(buyer)
    except:
        if buyer in unlisted_corp or buyer in people:
            continue
        elif len(re.findall(firm_suffix,buyer)) > 0:
            unlisted_corp.add(buyer)
        elif len(buyer)<=4 and len(re.findall(last_name,buyer)) > 0 :
            people.add(buyer)
        elif len(re.findall(r'^[^0-9A-Za-z]+-[0-9]+',buyer)) > 0:
            people.add(buyer)
        else:
            unlisted_corp.add(buyer)


In [19]:
n_listed_corp = len(listed_corp)
n_unlisted_corp = len(unlisted_corp)
n_people = len(people)

listed_corp = sorted(listed_corp)
people = sorted(people)
unlisted_corp = sorted(unlisted_corp)
print(n_listed_corp, n_unlisted_corp, n_people)
# 3月2日版本：4111 324764 212646

4111 324764 212646


写入 listed_firm.json，people.json，unlisted_firm.json

In [ ]:
'''with open('listed_firm.json','w') as f:
    json.dump(list(listed_corp),f,ensure_ascii=False)

with open('people.json','w') as f:
    json.dump(list(people),f,ensure_ascii=False)

with open('unlisted_firm.json','w') as f:
    json.dump(list(unlisted_corp),f,ensure_ascii=False)'''

In [ ]:
last_name = '赵、钱、孙、李、周、吴、郑、王、冯、陈、褚、卫、蒋、沈、韩、杨、朱、秦、尤、许、何、吕、施、张、孔、曹、严、华、金、魏、陶、姜、戚、谢、邹、喻、柏、水、窦、章、云、苏、潘、葛、奚、范、彭、郎、鲁、韦、昌、马、苗、凤、花、方、俞、任、袁、柳、酆、鲍、史、唐、费、廉、岑、薛、雷、贺、倪、汤、滕、殷、罗、毕、郝、邬、安、常、乐、于、时、傅、皮、卞、齐、康、伍、余、元、卜、顾、孟、平、黄、和、穆、萧、尹、姚、邵、湛、汪、祁、毛、禹、狄、米、贝、明、臧、计、伏、成、戴、谈、宋、茅、庞、熊、纪、舒、屈、项、祝、董、梁、杜、阮、蓝、闵、席、季、麻、强、贾、路、娄、危、江、童、颜、郭、梅、盛、林、刁、钟、徐、邱、骆、高、夏、蔡、田、樊、胡、凌、霍、虞、支、柯、昝、管、卢、莫、经、房、裘、缪、干、解、应、宗、丁、宣、贲、邓、郁、单、杭、洪、包、诸、左、石、崔、吉、钮、龚、程、嵇、邢、滑、裴、陆、荣、翁、荀、羊、於、惠、甄、麴、家、封、芮、羿、储、靳、汲、邴、糜、松、井、段、富、巫、乌、焦、巴、弓、牧、隗、山、谷、车、侯、宓、蓬、全、郗、班、仰、秋、仲、伊、宫、宁、仇、栾、暴、甘、钭、厉、戎、祖、武、符、刘、景、詹、束、叶、幸、司、韶、郜、黎、蓟、薄、印、宿、白、怀、蒲、邰、从、鄂、索、咸、籍、赖、卓、蔺、屠、蒙、池、乔、阴、郁、胥、能、苍、双、闻、莘、党、翟、谭、贡、劳、逄、姬、申、扶、堵冉、宰、郦、雍、舄、璩、桑、桂、濮、牛、寿、通、边、扈、燕、冀、郏、浦、尚、农、温、别、庄、晏、柴、瞿、阎、充、慕、连、茹、习、宦、艾、鱼、容、向、古、易、慎戈、廖、庾、终、暨、居、衡、步、都、耿、满、弘、匡、国、文、寇、广、禄、阙、东殴、殳、沃、利、蔚、越、夔、隆、师、巩、厍、聂、晁、勾、敖、融、冷、訾、辛、阚那、简、饶、空、曾、毋、沙、乜、养、鞠、须、丰、巢、关、蒯、相、查、後、荆、红游、竺、权、逯、盖、益、桓、公晋、楚、闫、法、汝、鄢、涂、钦岳、帅、缑、亢、况、后、有商、牟、佘、佴、伯、赏、南宫、墨、哈、谯、笪、年、爱、阳、佟言、福'
last_name = last_name.replace('、','')
# 查看 people 集合中还有多少是公司的名称
for i in people:
    if re.match(r'^[^0-9A-Za-z]+-[0-9]+',i) is None and re.match(r'[^a-zA-Z]+',i) is not None:
        if len(i)>3:
            print(i)

### 读取实体列表

In [22]:
with open('listed_firm.json','r') as f:
    listed_corp = json.load(f)
with open('unlisted_firm.json', 'r') as f:
    unlisted_corp = json.load(f)
with open('people.json','r') as f:
    people = json.load(f)

n_listed_corp = len(listed_corp)
n_unlisted_corp = len(unlisted_corp)
n_people = len(people)
n_all = n_listed_corp + n_unlisted_corp + n_people

print(n_listed_corp, n_unlisted_corp, n_people)

# 3月2日版本：4111 324764 212646
# 5月30日版本：4111 324764 212646，和3月2日一样

all_body = listed_corp + unlisted_corp + people
print(len(all_body))

# all_body_dict 用来找 entity 的下标
all_body_dict = {}
for i in range(len(all_body)):
    all_body_dict[all_body[i]] = i

4111 324764 212646
541521


### 计算 one hop 关系，结果和3.2那版不太一样

In [23]:
# 亲戚关系
# 构造matrix by year
all_body = listed_corp + unlisted_corp + people
n_qinshu = 0
_row, _col, _data= [], [], []
for year in range(2003,2022):
    for i in qinshu.index:
        cur_year = int(qinshu.loc[ i,'interval'])
        if cur_year != year:
            continue

        p1 = qinshu.loc[i,'person1']
        p2 = qinshu.loc[i,'person2']

        _row.append(all_body.index(p1))
        _col.append(all_body.index(p2))
        _data.append(1)

        _row.append(all_body.index(p2))
        _col.append(all_body.index(p1))
        _data.append(1)
    _row, _col, _data = np.array(_row), np.array(_col), np.array(_data)

    coo = coo_matrix((_data, (_row, _col)), shape= (n_all,n_all), dtype=np.int)
    n_qinshu += coo.nnz
    # mmwrite(os.path.join('mat_by_year', str(year), 'qinshu.mtx'),coo)
    _row, _col, _data = [], [], []
    
    print(year, coo.nnz, coo.shape,' qinshu done.')

2003 44 (541521, 541521)  qinshu done.
2004 156 (541521, 541521)  qinshu done.
2005 100 (541521, 541521)  qinshu done.
2006 118 (541521, 541521)  qinshu done.
2007 200 (541521, 541521)  qinshu done.
2008 334 (541521, 541521)  qinshu done.
2009 632 (541521, 541521)  qinshu done.
2010 1346 (541521, 541521)  qinshu done.
2011 1722 (541521, 541521)  qinshu done.
2012 1566 (541521, 541521)  qinshu done.
2013 1820 (541521, 541521)  qinshu done.
2014 1878 (541521, 541521)  qinshu done.
2015 2268 (541521, 541521)  qinshu done.
2016 2898 (541521, 541521)  qinshu done.
2017 4762 (541521, 541521)  qinshu done.
2018 3712 (541521, 541521)  qinshu done.
2019 3532 (541521, 541521)  qinshu done.
2020 3140 (541521, 541521)  qinshu done.
2021 3134 (541521, 541521)  qinshu done.


In [35]:
n_qin_by_year

[44,
 156,
 100,
 118,
 200,
 328,
 620,
 1336,
 1710,
 1560,
 1810,
 1870,
 2264,
 2870,
 4748,
 3708,
 3528,
 3138,
 3124]

In [30]:
# 高管关系
n_gaoguan = 0
# 构造matrix by year
all_body = listed_corp + unlisted_corp + people
_row, _col, _data = [], [], []
for year in range(2003,2022):
    for i in gaoguan.index:
        cur_year = int(gaoguan.loc[i,'interval'])
        if cur_year != year:
            continue
        stk = gaoguan.loc[i,'Stkcd']
        manager = gaoguan.loc[i,'manager']

        _row.append(listed_corp.index(stk))
        _col.append(all_body.index(manager))
        _data.append(1)

    _row, _col, _data = np.array(_row), np.array(_col), np.array(_data)

    coo = coo_matrix((_data, (_row, _col)), shape= (n_listed_corp, n_all), dtype=np.int)
    
    # mmwrite(os.path.join('mat_by_year', str(year), 'gaoguan.mtx'),coo)
    _row, _col, _data = [], [], []
    n_gaoguan += coo.nnz
    print(year, coo.nnz, coo.shape,'gaoguan done.')
print(n_gaoguan)

2003 21830 (4111, 541521) gaoguan done.
2004 23561 (4111, 541521) gaoguan done.
2005 23354 (4111, 541521) gaoguan done.
2006 24605 (4111, 541521) gaoguan done.


KeyboardInterrupt: 

In [ ]:
# 关联交易
_row, _col, _data = [], [], []
all_body = listed_corp + unlisted_corp + people
for year in range(2003,2022):
    for i in guanlian.index:
        cur_year = guanlian.loc[i,'interval']
        
        if year != cur_year:
            continue
        try:
            seller = int(guanlian.loc[i,'seller'])
        except:
            seller = guanlian.loc[i,'seller']
        try:
            buyer = int(guanlian.loc[i,'buyer'])
        except:
            buyer = guanlian.loc[i,'buyer']
        
        _row.append(all_body.index(seller))
        _col.append(all_body.index(buyer))
        _data.append(1)
        
        # 因为在计算元路径的时候，关联交易不管是买方是目标上市公司还是卖方是目标上市公司，都要算在一个矩阵里，这里做成对称的矩阵更好
        # 但是对称操作会使得有些数据重复，则导致 data 值大于1
        _row.append(all_body.index(buyer))
        _col.append(all_body.index(seller))
        _data.append(1)
    
    coo = coo_matrix((_data, (_row, _col)), shape = (len(all_body),len(all_body)), dtype=np.int)
    mmwrite(os.path.join('mat_by_year', str(year), 'guanlian.mtx'), coo)
    _row, _col, _data = [], [], []
    print(year, coo.nnz, coo.shape)


子公司关系并入股东关系里

In [32]:
# 子公司 & 股东关系

# 矩阵的 row 是股东， col是被持股方
_row, _col, _data = [], [], []
all_body = listed_corp + unlisted_corp + people

for year in range(2003,2022):
    # 子公司
    for i in subfirm.index:
        cur_year = int(subfirm.loc[i,'interval'])
        if cur_year != year:
            continue

        stkcd = subfirm.loc[i,'Stkcd']
        
        try: # 判断子公司是上市公司还是非上市公司
            sub = int(subfirm.loc[i,'subsidiary'])
        except:
            sub = subfirm.loc[i,'subsidiary']
        
        # row 是股东， col是被持股方
        _row.append(all_body_dict[stkcd])
        _col.append(all_body_dict[sub])
        _data.append(1)
    
    # 股东关系
    for i in gudong.index:
        cur_year = gudong.loc[i,'interval']
        if year != cur_year:
            continue
        holded = int(gudong.loc[i,'Stkcd'])
        try:
            holder = int(gudong.loc[i,'shareholder'])
        except:
            holder = gudong.loc[i,'shareholder']
        
        # row 是股东， col是被持股方
        _row.append(all_body_dict[holder])
        _col.append(all_body_dict[holded])
        _data.append(1)

    coo = coo_matrix((_data, (_row, _col)), shape= (len(all_body), len(all_body)), dtype=np.int)
    # coo.todense()  # 通过toarray方法转化成密集矩阵(numpy.matrix)
    mmwrite(os.path.join('mat_by_year', str(year), 'gud_&_sub.mtx'),coo)
    _row, _col, _data = [], [], []
    print(year, coo.nnz, coo.shape, 'gudong_subsidiary done.')

2003 22526 (541521, 541521) gudong_subsidiary done.
2004 24724 (541521, 541521) gudong_subsidiary done.
2005 25872 (541521, 541521) gudong_subsidiary done.
2006 28251 (541521, 541521) gudong_subsidiary done.
2007 30175 (541521, 541521) gudong_subsidiary done.
2008 32236 (541521, 541521) gudong_subsidiary done.
2009 35735 (541521, 541521) gudong_subsidiary done.
2010 39375 (541521, 541521) gudong_subsidiary done.
2011 47178 (541521, 541521) gudong_subsidiary done.
2012 51945 (541521, 541521) gudong_subsidiary done.
2013 61577 (541521, 541521) gudong_subsidiary done.
2014 71833 (541521, 541521) gudong_subsidiary done.
2015 84852 (541521, 541521) gudong_subsidiary done.
2016 93478 (541521, 541521) gudong_subsidiary done.
2017 117341 (541521, 541521) gudong_subsidiary done.
2018 126834 (541521, 541521) gudong_subsidiary done.
2019 140311 (541521, 541521) gudong_subsidiary done.
2020 165445 (541521, 541521) gudong_subsidiary done.
2021 171716 (541521, 541521) gudong_subsidiary done.


In [ ]:
# 子公司关系
''' 
_row, _col, _data = [], [], []
all_corp = listed_corp + unlisted_corp
all_body = listed_corp + unlisted_corp + people

# 构造matrix by year
for year in range(2003,2022):
    for i in subfirm.index:
        cur_year = int(subfirm.loc[i,'interval'])
        if cur_year != year:
            continue

        stkcd = subfirm.loc[i,'Stkcd']
        
        try: # 判断子公司是上市公司还是非上市公司
            sub = int(subfirm.loc[i,'subsidiary'])
        except:
            sub = subfirm.loc[i,'subsidiary']
        
        _row.append(listed_corp.index(stkcd))
        
        _col.append(all_corp.index(sub))
        
        _data.append(1)

    _row, _col, _data = np.array(_row), np.array(_col), np.array(_data)

    coo = coo_matrix((_data, (_row, _col)), shape= (n_listed_corp, n_listed_corp + n_unlisted_corp), dtype=np.int)
    # coo.todense()  # 通过toarray方法转化成密集矩阵(numpy.matrix)
    mmwrite(os.path.join('mat_by_year', str(year), 'subfirm.mtx'),coo)
    _row, _col, _data = [], [], []
    print(year, coo.nnz, coo.shape, 'subsidiary done.')
'''

In [ ]:
# 股东关系
# all_corp = listed_corp + unlisted_corp
'''
all_body = listed_corp + unlisted_corp + people

_row, _col, _data = [], [], []
for year in range(2003,2022):
    for i in gudong.index:
        cur_year = gudong.loc[i,'interval']
        if year != cur_year:
            continue
        holded = int(gudong.loc[i,'Stkcd'])
        try:
            holder = int(gudong.loc[i,'shareholder'])
        except:
            holder = gudong.loc[i,'shareholder']
        
        _row.append(listed_corp.index(holded))
        _col.append(all_body.index(holder))
        _data.append(1)
    coo = coo_matrix((_data, (_row, _col)), shape = (n_listed_corp, n_all), dtype=np.int)
    mmwrite(os.path.join('mat_by_year', str(year), 'gudong.mtx'),coo)
    _row, _col, _data = [], [], []
    # print(year,' gudong done.')
    print(year, coo.nnz, coo.shape)
'''

### 计算相似度

In [36]:
with open('listed_firm.json','r') as f:
    listed_corp = json.load(f)
with open('unlisted_firm.json', 'r') as f:
    unlisted_corp = json.load(f)
with open('people.json','r') as f:
    people = json.load(f)

n_listed_corp = len(listed_corp)
n_unlisted_corp = len(unlisted_corp)
n_people = len(people)
n_all = n_listed_corp + n_unlisted_corp + n_people

print(n_listed_corp, n_unlisted_corp, n_people, n_all)

def trans(D):
    return coo_matrix((D.data, (D.col, D.row)), shape=(D.shape[1], D.shape[0]))
# 3124 254457 160165 417746

4111 324764 212646 541521


In [3]:
def listed_row_slices(coo_mtx, n_listed_corp): # 对原始的矩阵进行行切片，以加快计算速度
    _row, _col, _data = [], [], []
    
    for row, col, data in zip(coo_mtx.row, coo_mtx.col, coo_mtx.data):
        if row < n_listed_corp:
            _row.append(row)
            _col.append(col)
            _data.append(data)
    return coo_matrix((_data, (_row, _col)), shape = (n_listed_corp, coo_mtx.shape[1]), dtype=np.int)

def listed_col_slices(coo_mtx, n_listed_corp): # 对原始的矩阵进行行切片，以加快计算速度
    _row, _col, _data = [], [], []
    
    for row, col, data in zip(coo_mtx.row, coo_mtx.col, coo_mtx.data):
        if col < n_listed_corp:
            _row.append(row)
            _col.append(col)
            _data.append(data)
    return coo_matrix((_data, (_row, _col)), shape = (coo_mtx.shape[0], n_listed_corp), dtype=np.int)

def listed_listed_slices(coo_mtx, n_listed_corp): # 对原始的矩阵进行行切片，以加快计算速度
    '''
    output: coo (4111, 4111)
    '''
    _row, _col, _data = [], [], []
    
    for row, col, data in zip(coo_mtx.row, coo_mtx.col, coo_mtx.data):
        if row < n_listed_corp and col < n_listed_corp:
            _row.append(row)
            _col.append(col)
            _data.append(data)
    return coo_matrix((_data, (_row, _col)), shape = (n_listed_corp, n_listed_corp), dtype=np.int)


统计路径从2003-2021年的总数

In [5]:
def sum_path_num(path_name):
    total_num = 0
    for year in range(2003, 2022):
        coo = mmread(f'mat_by_year/{year}/metapath/{path_name}.mtx')
        total_num+=coo.nnz
    return total_num
    
sum_path_num(path_name = 'gu(T)_qin_gu')

9869

### 长度为1的路径

listed_firm - [shareholderOf] - listed_firm

In [44]:
total_num = 0
for year in range(2003,2022):
    # [all_body, all_body]
    gudong = mmread(os.path.join('mat_by_year',str(year), 'gud_&_sub.mtx'))
    # [all_body, all_body]  -> [listed_corp, listed_corp]
    gudong = listed_listed_slices(gudong, n_listed_corp=n_listed_corp)
    
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/gudong.mtx'), gudong)
    total_num += gudong.nnz
    print(year, gudong.nnz, gudong.shape)
print(f'21年总共有: {total_num} 条路径')

2003 419 (4111, 4111)
2004 559 (4111, 4111)
2005 626 (4111, 4111)
2006 753 (4111, 4111)
2007 924 (4111, 4111)
2008 865 (4111, 4111)
2009 1291 (4111, 4111)
2010 1720 (4111, 4111)
2011 1739 (4111, 4111)
2012 1704 (4111, 4111)
2013 1860 (4111, 4111)
2014 4040 (4111, 4111)
2015 6496 (4111, 4111)
2016 5791 (4111, 4111)
2017 4656 (4111, 4111)
2018 3623 (4111, 4111)
2019 3782 (4111, 4111)
2020 4541 (4111, 4111)
2021 6000 (4111, 4111)
21年总共有: 51389 条路径


listed_firm - [HasTransaction] - listed_firm

In [45]:
total_num = 0
for year in range(2003,2022):
    # [all_body, all_body]
    guanlian = mmread(os.path.join('mat_by_year',str(year),'guanlian.mtx'))
    # [all_body, all_body]  -> [listed_corp, listed_corp]
    guanlian = listed_listed_slices(guanlian, n_listed_corp=n_listed_corp)
    
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/guanlian.mtx'), guanlian)
    total_num += guanlian.nnz
    print(year, guanlian.nnz, guanlian.shape)
print(f'21年总共有: {total_num} 条路径')

2003 258 (4111, 4111)
2004 232 (4111, 4111)
2005 248 (4111, 4111)
2006 307 (4111, 4111)
2007 314 (4111, 4111)
2008 426 (4111, 4111)
2009 428 (4111, 4111)
2010 520 (4111, 4111)
2011 613 (4111, 4111)
2012 662 (4111, 4111)
2013 721 (4111, 4111)
2014 821 (4111, 4111)
2015 926 (4111, 4111)
2016 1051 (4111, 4111)
2017 1218 (4111, 4111)
2018 1302 (4111, 4111)
2019 1565 (4111, 4111)
2020 1686 (4111, 4111)
2021 1797 (4111, 4111)
21年总共有: 15095 条路径


### 长度为2的路径

In [15]:
import meta_2rel_v2 # v2 太慢了

1. (C1, shareholderOf -1, C3, shareholderOf, C2)
2. (C1, shareholderOf -1, C3, hasTransaction, C2)
3. (C1, shareholderOf-1, P, managerOf, C2)
4. (C1, shareholderOf -1, C3, shareholderOf-1, C2) (C1, shareholderOf , C3, shareholderOf, C2)
5. (C1, hasTransaction, C3, hasTransaction, C2)
6. (C1, hasTransaction, P, managerOf, C2)
7. (C1, shareholderOf, C3, hasTransaction, C2)
8. (C1, managerOf -1, P, managerOf, C2)
9. (C1, shareholderOf, C3, shareholderOf-1, C2)

1. (C1, shareholderOf-1, C3, shareholderOf, C2)


In [50]:
total_num = 0
for year in range(2003,2022):
    # [all_body, all_body]
    gudong = mmread(os.path.join('mat_by_year',str(year), 'gud_&_sub.mtx'))
    # [all_body, listed_corp] 行是 股东，列是被持股方
    gudong = listed_col_slices(gudong, n_listed_corp)
    # [listed_corp, all_body]
    gudong_T = trans(gudong)

    gudT_gud = meta_2rel_v2.cal_metapath(gudong_T, gudong, del_diagnoal=True)
    print(year, gudT_gud.nnz, gudT_gud.shape)
    total_num += gudT_gud.nnz

    mmwrite(os.path.join('mat_by_year',str(year),'metapath/gud(T)_gud.mtx'), gudT_gud)

print(f'21年总共有: {total_num} 条路径') # 16812096

2003 7656 (4111, 4111)
2004 17102 (4111, 4111)
2005 19036 (4111, 4111)
2006 30238 (4111, 4111)
2007 45686 (4111, 4111)
2008 41166 (4111, 4111)
2009 89640 (4111, 4111)
2010 164676 (4111, 4111)
2011 163172 (4111, 4111)
2012 157724 (4111, 4111)
2013 167624 (4111, 4111)
2014 1135048 (4111, 4111)
2015 3011894 (4111, 4111)
2016 2250474 (4111, 4111)
2017 1468782 (4111, 4111)
2018 1124826 (4111, 4111)
2019 1657594 (4111, 4111)
2020 2287526 (4111, 4111)
2021 2972232 (4111, 4111)
21年总共有: 16812096 条路径


4. (C1, shareholderOf -1, C3, shareholderOf-1, C2) or (C1, shareholderOf , C3, shareholderOf, C2), 这里用后者来计算

In [51]:
total_num = 0
for year in range(2003,2022):
    # [all_body, all_body]
    gudong = mmread(os.path.join('mat_by_year',str(year), 'gud_&_sub.mtx'))
    # [listed_corp, all_body] 行是股东，列是被持股方
    gudong = listed_row_slices(gudong, n_listed_corp)
    
    # [all_body, listed_corp] 行是股东，列是被持股方
    gudong_level2 = listed_col_slices(gudong, n_listed_corp)
    
    gud_gud = meta_2rel_v2.cal_metapath(gudong, gudong_level2, del_diagnoal=True)
    print(year, gud_gud.nnz, gud_gud.shape)
    total_num += gud_gud.nnz

    mmwrite(os.path.join('mat_by_year',str(year),'metapath/gud_gud.mtx'), gud_gud)

print(f'21年总共有: {total_num} 条路径') # 7725

2003 99 (4111, 4111)
2004 162 (4111, 4111)
2005 160 (4111, 4111)
2006 202 (4111, 4111)
2007 173 (4111, 4111)
2008 149 (4111, 4111)
2009 245 (4111, 4111)
2010 272 (4111, 4111)
2011 243 (4111, 4111)
2012 302 (4111, 4111)
2013 293 (4111, 4111)
2014 459 (4111, 4111)
2015 697 (4111, 4111)
2016 741 (4111, 4111)
2017 691 (4111, 4111)
2018 583 (4111, 4111)
2019 552 (4111, 4111)
2020 692 (4111, 4111)
2021 1010 (4111, 4111)
21年总共有: 7725 条路径


9. (C1, shareholderOf, C3, shareholderOf-1, C2)

In [52]:
total_num = 0
for year in range(2003,2022):
    # [all_body, all_body]
    gudong = mmread(os.path.join('mat_by_year',str(year), 'gud_&_sub.mtx'))
    # [listed_corp, all_body] 行是股东，列是被持股方
    gudong = listed_row_slices(gudong, n_listed_corp)
    
    gudong_T = trans(gudong)
    
    gud_gudT = meta_2rel_v2.cal_metapath(gudong, gudong_T, del_diagnoal=True)
    print(year, gud_gudT.nnz, gud_gudT.shape)
    total_num += gud_gudT.nnz

    mmwrite(os.path.join('mat_by_year',str(year),'metapath/gud_gud(T).mtx'), gud_gudT)

print(f'21年总共有: {total_num} 条路径') # 23292

2003 212 (4111, 4111)
2004 326 (4111, 4111)
2005 380 (4111, 4111)
2006 496 (4111, 4111)
2007 518 (4111, 4111)
2008 478 (4111, 4111)
2009 730 (4111, 4111)
2010 864 (4111, 4111)
2011 832 (4111, 4111)
2012 798 (4111, 4111)
2013 892 (4111, 4111)
2014 1474 (4111, 4111)
2015 2436 (4111, 4111)
2016 2426 (4111, 4111)
2017 2128 (4111, 4111)
2018 1718 (4111, 4111)
2019 1550 (4111, 4111)
2020 2008 (4111, 4111)
2021 3026 (4111, 4111)
21年总共有: 23292 条路径


2. (C1, shareholderOf -1, C3, hasTransaction, C2)

In [55]:
total_num = 0
for year in range(2003,2022):
    # [all_body, all_body]
    gudong = mmread(os.path.join('mat_by_year',str(year), 'gud_&_sub.mtx'))
    # [all_body, listed_corp] 行是股东，列是被持股方
    gudong = listed_col_slices(gudong, n_listed_corp)
    # [listed_corp, all_body]
    gudong_T = trans(gudong)
    # [all_body, all_body]
    guanlian = mmread(os.path.join('mat_by_year',str(year),'guanlian.mtx'))
    # [all_body, listed_corp]
    guanlian = listed_col_slices(guanlian, n_listed_corp=n_listed_corp)
    
    gudT_trans = meta_2rel_v2.cal_metapath(gudong_T, guanlian, del_diagnoal=False)

    mmwrite(f'mat_by_year/{year}/metapath/gud(T)_trans.mtx', gudT_trans)
    print(year, gudT_trans.nnz, gudT_trans.shape)
    total_num += gudT_trans.nnz
print(f'21年总共有: {total_num} 条路径') # 89353

2003 1238 (4111, 4111)
2004 1533 (4111, 4111)
2005 1593 (4111, 4111)
2006 1863 (4111, 4111)
2007 1912 (4111, 4111)
2008 2148 (4111, 4111)
2009 2471 (4111, 4111)
2010 2514 (4111, 4111)
2011 3101 (4111, 4111)
2012 3199 (4111, 4111)
2013 3633 (4111, 4111)
2014 4428 (4111, 4111)
2015 5748 (4111, 4111)
2016 5598 (4111, 4111)
2017 6158 (4111, 4111)
2018 7612 (4111, 4111)
2019 9060 (4111, 4111)
2020 11580 (4111, 4111)
2021 13964 (4111, 4111)
21年总共有: 89353 条路径


7. (C1, shareholderOf, C3, hasTransaction, C2)

In [72]:
total_num = 0
for year in range(2021,2022):
    # [all_body, all_body]
    gudong = mmread(os.path.join('mat_by_year',str(year), 'gud_&_sub.mtx'))
    # [listed_corp, all_body] 行是股东，列是被持股方
    gudong = listed_row_slices(gudong, n_listed_corp)
    
    # [all_body, all_body]
    guanlian = mmread(os.path.join('mat_by_year',str(year),'guanlian.mtx'))
    # [all_body, listed_corp]
    guanlian = listed_col_slices(guanlian, n_listed_corp=n_listed_corp)
    
    gud_trans = meta_2rel_v2.cal_metapath(gudong, guanlian, del_diagnoal=False)

    mmwrite(f'mat_by_year/{year}/metapath/gud_trans.mtx', gud_trans)
    print(year, gud_trans.nnz, gud_trans.shape)
    total_num += gud_trans.nnz
print(f'21年总共有: {total_num} 条路径') # 121334

KeyboardInterrupt: 

3. (C1, shareholderOf-1, P, managerOf, C2)

In [62]:
total_num = 0
for year in range(2003,2022):
    # [all_body, all_body]
    gudong = mmread(os.path.join('mat_by_year',str(year), 'gud_&_sub.mtx'))
    # [all_body, listed_corp] 行是股东，列是被持股方
    gudong = listed_col_slices(gudong, n_listed_corp)
    # [listed_corp, all_body]
    gudong_T = trans(gudong)

    # [listed_corp, all_body]
    gaoguan = mmread(os.path.join('mat_by_year',str(year),'gaoguan.mtx'))
    gaoguan_T = trans(gaoguan)
    gudT_gaoguan = meta_2rel_v2.cal_metapath(gudong_T, gaoguan_T, del_diagnoal=False)
    
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/gud(T)_mana.mtx'), gudT_gaoguan)
    print(year, gudT_gaoguan.nnz, gudT_gaoguan.shape)
    total_num += gudT_gaoguan.nnz

print(f'21年总共有: {total_num} 条路径') # 27776


2003 102 (4111, 4111)
2004 154 (4111, 4111)
2005 166 (4111, 4111)
2006 234 (4111, 4111)
2007 341 (4111, 4111)
2008 392 (4111, 4111)
2009 552 (4111, 4111)
2010 865 (4111, 4111)
2011 1119 (4111, 4111)
2012 1260 (4111, 4111)
2013 1532 (4111, 4111)
2014 1784 (4111, 4111)
2015 2135 (4111, 4111)
2016 2289 (4111, 4111)
2017 2705 (4111, 4111)
2018 2775 (4111, 4111)
2019 3096 (4111, 4111)
2020 3094 (4111, 4111)
2021 3181 (4111, 4111)
21年总共有: 27776 条路径


5. (C1, hasTransaction, C3, hasTransaction, C2)

In [63]:
total_num = 0
for year in range(2003,2022):
    # [all_corp, all_body]
    guanlian = mmread(os.path.join('mat_by_year',str(year),'guanlian.mtx'))
    # [listed_corp, all_body]
    guanlian = listed_row_slices(guanlian, n_listed_corp=n_listed_corp)
    guanlian_T = trans(guanlian)
    
    trans_trans = meta_2rel_v2.cal_metapath(guanlian, guanlian_T, del_diagnoal=False)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/trans_trans.mtx'), trans_trans)
    print(year, trans_trans.nnz, trans_trans.shape)
    total_num += trans_trans.nnz

print(f'21年总共有: {total_num} 条路径') # 89019


2003 1424 (4111, 4111)
2004 1632 (4111, 4111)
2005 1749 (4111, 4111)
2006 1867 (4111, 4111)
2007 2055 (4111, 4111)
2008 2493 (4111, 4111)
2009 2872 (4111, 4111)
2010 3200 (4111, 4111)
2011 3694 (4111, 4111)
2012 4105 (4111, 4111)
2013 4484 (4111, 4111)
2014 4858 (4111, 4111)
2015 5362 (4111, 4111)
2016 6126 (4111, 4111)
2017 7112 (4111, 4111)
2018 7710 (4111, 4111)
2019 8814 (4111, 4111)
2020 9418 (4111, 4111)
2021 10044 (4111, 4111)
21年总共有: 89019 条路径


In [65]:
print(f'21年总共有: {total_num} 条路径') # 89019

21年总共有: 89019 条路径


6. (C1, hasTransaction, P, managerOf, C2)

In [66]:
total_num = 0
for year in range(2003,2022):
    # [all_corp, all_body]
    guanlian = mmread(os.path.join('mat_by_year',str(year),'guanlian.mtx'))
    # [listed_corp, all_body]
    guanlian = listed_row_slices(guanlian, n_listed_corp=n_listed_corp)
    # [listed_corp, all_body]
    gaoguan = mmread(os.path.join('mat_by_year',str(year),'gaoguan.mtx'))
    gaoguan_T = trans(gaoguan)
    
    trans_mana = meta_2rel_v2.cal_metapath(guanlian, gaoguan_T, del_diagnoal=False)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/trans_mana.mtx'), trans_mana)
    print(year, trans_mana.nnz, trans_mana.shape)
    total_num += trans_mana.nnz

print(f'21年总共有: {total_num} 条路径') # 10250 

2003 6 (4111, 4111)
2004 12 (4111, 4111)
2005 23 (4111, 4111)
2006 165 (4111, 4111)
2007 83 (4111, 4111)
2008 186 (4111, 4111)
2009 282 (4111, 4111)
2010 293 (4111, 4111)
2011 387 (4111, 4111)
2012 410 (4111, 4111)
2013 455 (4111, 4111)
2014 707 (4111, 4111)
2015 859 (4111, 4111)
2016 995 (4111, 4111)
2017 1064 (4111, 4111)
2018 1108 (4111, 4111)
2019 1144 (4111, 4111)
2020 1076 (4111, 4111)
2021 995 (4111, 4111)
21年总共有: 10250 条路径


8. (C1, managerOf -1, P, managerOf, C2)

In [67]:
total_num = 0
for year in range(2003,2022):
    # [listed_corp, all_body]
    gaoguan = mmread(os.path.join('mat_by_year',str(year),'gaoguan.mtx'))
    gaoguan_T = trans(gaoguan)
    mana_mana = meta_2rel_v2.cal_metapath(gaoguan, gaoguan_T, del_diagnoal=False)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/mana_mana.mtx'), mana_mana)
    print(year, mana_mana.nnz, mana_mana.shape)
    total_num += mana_mana.nnz

print(f'21年总共有: {total_num} 条路径') # 247012

2003 4335 (4111, 4111)
2004 5010 (4111, 4111)
2005 4927 (4111, 4111)
2006 5202 (4111, 4111)
2007 5902 (4111, 4111)
2008 6564 (4111, 4111)
2009 7377 (4111, 4111)
2010 9640 (4111, 4111)
2011 11394 (4111, 4111)
2012 12728 (4111, 4111)
2013 14067 (4111, 4111)
2014 16102 (4111, 4111)
2015 16945 (4111, 4111)
2016 19553 (4111, 4111)
2017 21869 (4111, 4111)
2018 20761 (4111, 4111)
2019 21920 (4111, 4111)
2020 21765 (4111, 4111)
2021 20951 (4111, 4111)
21年总共有: 247012 条路径


In [68]:
total_num

247012

### 长度为3的路径

1. (C1, shareholderOf-1, C3, kinship, C4, shareholderOf, C2)
2. (C1, shareholderOf-1, C3, kinship, C4, hasTransaction, C2)
3. (C1, shareholderOf-1, C3, kinship, C4, managerOf, C2)
4. (C1, hasTransaction, C3, kinship, C4, hasTransaction, C2)
5. (C1, hasTransaction, C3, kinship, C4, managerOf, C2)
6. (C1, managerOf-1, C3, kinship, C4, managerOf, C2)

In [6]:
import meta_3rel_v2
# gudong: (listed_corp + unlisted_corp) * (listed_corp + unlisted_corp + people)

# first_quadrant 切到第一象限: 行往小了切，列往大了切
def first_quadrant_slices(coo_mtx, upper_bound, lower_bound):
    _row, _col, _data = [], [], []
    for r, c, d in zip(coo_mtx.row, coo_mtx.col, coo_mtx.data):
        if r < upper_bound and c >= lower_bound:
            _row.append(r)
            _col.append(c-lower_bound)
            _data.append(d)
    return coo_matrix((_data, (_row, _col)), shape = (upper_bound, coo_mtx.shape[1]-lower_bound), dtype=np.int)

1. (C1, shareholderOf-1, C3, kinship, C4, shareholderOf, C2)

In [116]:
total_num = 0
for year in range(2003,2022):
    # [all_body, all_body]
    qinshu = mmread(os.path.join('mat_by_year', str(year), 'qinshu.mtx'))
    # [all_body, all_body]
    gudong = mmread(os.path.join('mat_by_year',str(year), 'gud_&_sub.mtx'))
    # [all_body, listed_corp] 行是 股东，列是被持股方
    gudong = listed_col_slices(gudong, n_listed_corp)
    # [listed_corp, all_body]
    gudong_T = trans(gudong)
    guT_qin_gu = meta_3rel_v2.cal_metapath(gudong_T, qinshu, gudong)
    
    print(year,guT_qin_gu.nnz, guT_qin_gu.shape)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/gu(T)_qin_gu.mtx'),guT_qin_gu)
    total_num += guT_qin_gu.nnz
print(f'21年总共有: {total_num} 条路径') # 27018

2003 3 (4111, 4111)
2004 11 (4111, 4111)
2005 15 (4111, 4111)
2006 24 (4111, 4111)
2007 51 (4111, 4111)
2008 60 (4111, 4111)
2009 106 (4111, 4111)
2010 207 (4111, 4111)
2011 252 (4111, 4111)
2012 283 (4111, 4111)
2013 325 (4111, 4111)
2014 686 (4111, 4111)
2015 1212 (4111, 4111)
2016 991 (4111, 4111)
2017 1167 (4111, 4111)
2018 934 (4111, 4111)
2019 1015 (4111, 4111)
2020 1105 (4111, 4111)
2021 1422 (4111, 4111)


2. (C1, shareholderOf-1, C3, kinship, C4, hasTransaction, C2)

In [118]:
total_num = 0
for year in range(2003,2022):
    # [all_body, all_body]
    qinshu = mmread(os.path.join('mat_by_year', str(year), 'qinshu.mtx'))
    # [all_body, all_body]
    gudong = mmread(os.path.join('mat_by_year',str(year), 'gud_&_sub.mtx'))
    # [all_body, listed_corp] 行是 股东，列是被持股方
    gudong = listed_col_slices(gudong, n_listed_corp)
    gudong_T = trans(gudong)
    
    # [all_body, all_body]
    guanlian = mmread(os.path.join('mat_by_year',str(year),'guanlian.mtx'))
    # [all_body, listed_corp]
    guanlian = listed_col_slices(guanlian, n_listed_corp)
    guT_qin_trans = meta_3rel_v2.cal_metapath(gudong_T, qinshu, guanlian)
    
    print(year,guT_qin_trans.nnz, guT_qin_trans.shape)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/gu(T)_qin_trans.mtx'),guT_qin_trans)
    total_num += guT_qin_trans.nnz
print(f'21年总共有: {total_num} 条路径') # 2715

2003 1 (4111, 4111)
2004 3 (4111, 4111)
2005 5 (4111, 4111)
2006 9 (4111, 4111)
2007 20 (4111, 4111)
2008 25 (4111, 4111)
2009 40 (4111, 4111)
2010 60 (4111, 4111)
2011 87 (4111, 4111)
2012 93 (4111, 4111)
2013 107 (4111, 4111)
2014 168 (4111, 4111)
2015 239 (4111, 4111)
2016 266 (4111, 4111)
2017 309 (4111, 4111)
2018 356 (4111, 4111)
2019 365 (4111, 4111)
2020 288 (4111, 4111)
2021 274 (4111, 4111)
21年总共有: 2715 条路径


3. (C1, shareholderOf-1, C3, kinship, C4, managerOf, C2)

In [123]:
total_num = 0
for year in range(2003,2022):
    # [all_body, all_body]
    qinshu = mmread(os.path.join('mat_by_year', str(year), 'qinshu.mtx'))
    # [all_body, all_body]
    gudong = mmread(os.path.join('mat_by_year',str(year), 'gud_&_sub.mtx'))
    # [all_body, listed_corp] 行是 股东，列是被持股方
    gudong = listed_col_slices(gudong, n_listed_corp)
    gudong_T = trans(gudong)
    
    # [listed_corp, all_body]
    gaoguan = mmread(os.path.join('mat_by_year',str(year), 'gaoguan.mtx'))
    gaoguan_T = trans(gaoguan)
    
    
    guT_qin_mana = meta_3rel_v2.cal_metapath(gudong_T, qinshu, gaoguan_T)
    
    print(year,guT_qin_mana.nnz, guT_qin_mana.shape)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/gu(T)_qin_mana.mtx'),guT_qin_mana)
    total_num += guT_qin_mana.nnz

print(f'21年总共有: {total_num} 条路径')

2003 4 (4111, 4111)
2004 20 (4111, 4111)
2005 17 (4111, 4111)
2006 28 (4111, 4111)
2007 49 (4111, 4111)
2008 62 (4111, 4111)
2009 105 (4111, 4111)
2010 211 (4111, 4111)
2011 287 (4111, 4111)
2012 320 (4111, 4111)
2013 358 (4111, 4111)
2014 412 (4111, 4111)
2015 509 (4111, 4111)
2016 638 (4111, 4111)
2017 826 (4111, 4111)
2018 828 (4111, 4111)
2019 861 (4111, 4111)
2020 774 (4111, 4111)
2021 791 (4111, 4111)
21年总共有: 7100 条路径


4. (C1, hasTransaction, C3, kinship, C4, hasTransaction, C2)

In [124]:
total_num = 0
for year in range(2003,2022):
    # [all_body, all_body]
    qinshu = mmread(os.path.join('mat_by_year', str(year), 'qinshu.mtx'))
    # [all_body, all_body]
    guanlian = mmread(os.path.join('mat_by_year',str(year),'guanlian.mtx'))
    # [listed_corp, all_body]
    guanlian = listed_row_slices(guanlian, n_listed_corp)
    guanlian_T = trans(guanlian)
    trans_qin_trans = meta_3rel_v2.cal_metapath(guanlian, qinshu, guanlian_T)
    
    print(year,trans_qin_trans.nnz, trans_qin_trans.shape)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/trans_qin_trans.mtx'),trans_qin_trans)
    total_num += trans_qin_trans.nnz

print(f'21年总共有: {total_num} 条路径') # 449

2003 0 (4111, 4111)
2004 1 (4111, 4111)
2005 1 (4111, 4111)
2006 3 (4111, 4111)
2007 6 (4111, 4111)
2008 4 (4111, 4111)
2009 7 (4111, 4111)
2010 13 (4111, 4111)
2011 13 (4111, 4111)
2012 13 (4111, 4111)
2013 20 (4111, 4111)
2014 37 (4111, 4111)
2015 44 (4111, 4111)
2016 56 (4111, 4111)
2017 48 (4111, 4111)
2018 47 (4111, 4111)
2019 56 (4111, 4111)
2020 44 (4111, 4111)
2021 36 (4111, 4111)
21年总共有: 449 条路径


5. (C1, hasTransaction, C3, kinship, C4, managerOf, C2)

In [122]:
total_num = 0
for year in range(2003,2022):
    # [all_body, all_body]
    qinshu = mmread(os.path.join('mat_by_year', str(year), 'qinshu.mtx'))
    # [all_body, all_body]
    guanlian = mmread(os.path.join('mat_by_year',str(year),'guanlian.mtx'))
    # [listed_corp, all_body]
    guanlian = listed_row_slices(guanlian, n_listed_corp)
    # [listed_corp, all_body]
    gaoguan = mmread(os.path.join('mat_by_year',str(year), 'gaoguan.mtx'))
    gaoguan_T = trans(gaoguan)
    
    trans_qin_mana = meta_3rel_v2.cal_metapath(guanlian, qinshu, gaoguan_T)
    
    print(year,trans_qin_mana.nnz, trans_qin_mana.shape)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/trans_qin_mana.mtx'),trans_qin_mana)
    total_num += trans_qin_mana.nnz

print(f'21年总共有: {total_num} 条路径') # 1995

2003 1 (4111, 4111)
2004 3 (4111, 4111)
2005 6 (4111, 4111)
2006 8 (4111, 4111)
2007 15 (4111, 4111)
2008 23 (4111, 4111)
2009 35 (4111, 4111)
2010 62 (4111, 4111)
2011 85 (4111, 4111)
2012 85 (4111, 4111)
2013 93 (4111, 4111)
2014 126 (4111, 4111)
2015 171 (4111, 4111)
2016 190 (4111, 4111)
2017 221 (4111, 4111)
2018 241 (4111, 4111)
2019 244 (4111, 4111)
2020 204 (4111, 4111)
2021 182 (4111, 4111)
21年总共有: 1995 条路径


### 用同一个函数来计算 3-hop metapath

In [6]:
name_path_dict = {
    'gud': 'gud_&_sub.mtx',
    'gu': 'gud_&_sub.mtx',
    'trans': 'guanlian.mtx',
    'mana': 'gaoguan.mtx',
    'qin': 'qinshu.mtx',
}

def cal_3hop_metapath(PATH_NAME):
    # PATH_NAME: trans_qin_mana
    walks = PATH_NAME.split('_')
    
    IS_TRANSFER = [True if '(T)' in w else False for w in walks]
    # 去掉逆标记
    walks = [w.replace('(T)', '') for w in walks]
    total_num = 0
    
    for year in range(2003,2022):
        mat_1 = mmread(os.path.join('mat_by_year',str(year), name_path_dict[walks[0]]))
        mat_2 = mmread(os.path.join('mat_by_year',str(year), name_path_dict[walks[1]]))
        mat_3 = mmread(os.path.join('mat_by_year',str(year), name_path_dict[walks[2]]))
        '''
        mat_1
        '''
        # 有可能是 shareholderOf，managerOf，hasTransaction。其中 hasTransaction 不可能有逆
        if walks[0]=='gud' or walks[0]=='gu':
            if IS_TRANSFER[0] is True: # True
                # [all_body, listed_corp]
                mat_1 = listed_col_slices(mat_1, n_listed_corp)
                mat_1 = trans(mat_1)
            else:
                # [listed_corp, all_body]
                mat_1 = listed_row_slices(mat_1, n_listed_corp)
        elif walks[0]=='mana': # mana 位于第一个walk 时，必然有 (T), 不用切片，也不用逆，直接读进来就是 [listed_corp, all_body]
            pass
        elif walks[0]=='trans':
            mat_1 = listed_row_slices(mat_1, n_listed_corp)
        '''
        mat_2
        '''
        # 可能是 shareholderOf，managerOf，hasTransaction, kinship。其中 hasTransaction, kinship 不可能有逆，
        # mat_2 不管是什么关系，都不需要对 row 或者 col 切分
        if walks[1] == 'mana': # mana 和其他的相反，有(T) 不需要逆，没有 (T) 才需要逆
            if  IS_TRANSFER[1] is False: 
                mat_2 = trans(mat_2)
        else: # 其他关系正常：有(T)就取逆
            if IS_TRANSFER[1] is True: 
                # [all_body, all_body]
                mat_2 = trans(mat_2)
        '''
        mat_3
        '''
        if walks[2]=='gud' or walks[2]=='gu':
            if IS_TRANSFER[2] is True: 
                # 上市公司是股东，即subsidiaryOf
                mat_3 = listed_row_slices(mat_3, n_listed_corp)
                mat_3 = trans(mat_3)
            else:
                # 上市公司是子公司，即shareholderOf
                mat_3 = listed_col_slices(mat_3, n_listed_corp)
        elif walks[2]=='trans':
            mat_3 = listed_col_slices(mat_3, n_listed_corp)
        elif walks[2]=='mana':
            # mana 必然没有(T), 和其他关系是相反的，没有(T)就要取逆，即 从 [listed_corp, all_body] -> [all_body, listed_corp]
            mat_3 = trans(mat_3)

        meta_path = meta_3rel_v2.cal_metapath(mat_1, mat_2, mat_3)
    
        print(year,meta_path.nnz, meta_path.shape)
        mmwrite(os.path.join('mat_by_year',str(year),f'metapath/{PATH_NAME}.mtx'),meta_path)
        total_num += meta_path.nnz

    print(f'{PATH_NAME} 2003-2021年总共有: {total_num} 条路径')
    return total_num

6. (C1, managerOf-1, C3, kinship, C4, managerOf, C2)

In [152]:
total_num = cal_3hop_metapath(PATH_NAME='mana(T)_qin_mana') # 4575

['mana', 'qin', 'mana'] [True, False, False]
2003 9 (4111, 4111)
2004 31 (4111, 4111)
2005 13 (4111, 4111)
2006 24 (4111, 4111)
2007 33 (4111, 4111)
2008 44 (4111, 4111)
2009 88 (4111, 4111)
2010 194 (4111, 4111)
2011 258 (4111, 4111)
2012 263 (4111, 4111)
2013 282 (4111, 4111)
2014 275 (4111, 4111)
2015 346 (4111, 4111)
2016 404 (4111, 4111)
2017 480 (4111, 4111)
2018 489 (4111, 4111)
2019 479 (4111, 4111)
2020 433 (4111, 4111)
2021 430 (4111, 4111)
mana(T)_qin_mana 2003-2021年总共有: 4575 条路径


#### 包含managerOf，不包含kinshipOf:
1. ManagerOf-1, HasTransaction, HasTransaction
2. HasTransaction, ManagerOf-1, HasTransaction
3. ShareholderOf, HasTransaction, ManagerOf
4. ShareholderOf-1, HasTransaction, ManagerOf
5. HasTransaction, ShareholderOf-1, ManagerOf
6. ShareholderOf-1, ManagerOf, HasTransaction
7. HasTransaction, ManagerOf, ShareholderOf-1
8. ManagerOf-1, ManagerOf, HasTransaction
9. ManagerOf-1, ManagerOf, ShareholderOf
10. ManagerOf-1, ManagerOf, ShareholderOf-1

1. ManagerOf-1, HasTransaction, HasTransaction

In [ ]:
total_num = cal_3hop_metapath(PATH_NAME='mana(T)_trans_trans') # 2419

2. HasTransaction, ManagerOf-1, HasTransaction

In [11]:
total_num = cal_3hop_metapath(PATH_NAME='trans_mana(T)_trans') # 1706

2003 0 (4111, 4111)
2004 1 (4111, 4111)
2005 4 (4111, 4111)
2006 35 (4111, 4111)
2007 5 (4111, 4111)
2008 45 (4111, 4111)
2009 68 (4111, 4111)
2010 38 (4111, 4111)
2011 46 (4111, 4111)
2012 52 (4111, 4111)
2013 67 (4111, 4111)
2014 107 (4111, 4111)
2015 139 (4111, 4111)
2016 151 (4111, 4111)
2017 152 (4111, 4111)
2018 174 (4111, 4111)
2019 204 (4111, 4111)
2020 208 (4111, 4111)
2021 210 (4111, 4111)
trans_mana(T)_trans 2003-2021年总共有: 1706 条路径


3. ShareholderOf, HasTransaction, ManagerOf


In [12]:
total_num = cal_3hop_metapath(PATH_NAME='gud_trans_mana') # 12512, 107min

2003 0 (4111, 4111)
2004 7 (4111, 4111)
2005 8 (4111, 4111)
2006 64 (4111, 4111)
2007 39 (4111, 4111)
2008 114 (4111, 4111)
2009 198 (4111, 4111)
2010 193 (4111, 4111)
2011 225 (4111, 4111)
2012 334 (4111, 4111)
2013 370 (4111, 4111)
2014 1109 (4111, 4111)
2015 2081 (4111, 4111)
2016 1709 (4111, 4111)
2017 1240 (4111, 4111)
2018 992 (4111, 4111)
2019 1066 (4111, 4111)
2020 1273 (4111, 4111)
2021 1490 (4111, 4111)
gud_trans_mana 2003-2021年总共有: 12512 条路径


In [13]:
total_num

12512

4. ShareholderOf-1, HasTransaction, ManagerOf

In [7]:
total_num = cal_3hop_metapath(PATH_NAME='gud(T)_trans_mana') # 1078, 52min

2003 0 (4111, 4111)
2004 0 (4111, 4111)
2005 0 (4111, 4111)
2006 18 (4111, 4111)
2007 1 (4111, 4111)
2008 18 (4111, 4111)
2009 14 (4111, 4111)
2010 39 (4111, 4111)
2011 51 (4111, 4111)
2012 20 (4111, 4111)
2013 44 (4111, 4111)
2014 49 (4111, 4111)
2015 108 (4111, 4111)
2016 133 (4111, 4111)
2017 90 (4111, 4111)
2018 109 (4111, 4111)
2019 123 (4111, 4111)
2020 129 (4111, 4111)
2021 132 (4111, 4111)
gud(T)_trans_mana 2003-2021年总共有: 1078 条路径


5. HasTransaction, ShareholderOf-1, ManagerOf


In [8]:
total_num = cal_3hop_metapath(PATH_NAME='trans_gud(T)_mana') # 5983

2003 13 (4111, 4111)
2004 22 (4111, 4111)
2005 26 (4111, 4111)
2006 42 (4111, 4111)
2007 42 (4111, 4111)
2008 61 (4111, 4111)
2009 73 (4111, 4111)
2010 118 (4111, 4111)
2011 160 (4111, 4111)
2012 180 (4111, 4111)
2013 237 (4111, 4111)
2014 324 (4111, 4111)
2015 490 (4111, 4111)
2016 502 (4111, 4111)
2017 584 (4111, 4111)
2018 619 (4111, 4111)
2019 823 (4111, 4111)
2020 812 (4111, 4111)
2021 855 (4111, 4111)
trans_gud(T)_mana 2003-2021年总共有: 5983 条路径


In [9]:
print(total_num)

5983


6. ShareholderOf-1, ManagerOf, HasTransaction

In [10]:
total_num = cal_3hop_metapath(PATH_NAME='gud(T)_mana_trans')
print(total_num) # 6550

2003 16 (4111, 4111)
2004 26 (4111, 4111)
2005 21 (4111, 4111)
2006 38 (4111, 4111)
2007 50 (4111, 4111)
2008 72 (4111, 4111)
2009 95 (4111, 4111)
2010 140 (4111, 4111)
2011 189 (4111, 4111)
2012 195 (4111, 4111)
2013 263 (4111, 4111)
2014 316 (4111, 4111)
2015 424 (4111, 4111)
2016 482 (4111, 4111)
2017 626 (4111, 4111)
2018 678 (4111, 4111)
2019 885 (4111, 4111)
2020 947 (4111, 4111)
2021 1087 (4111, 4111)
gud(T)_mana_trans 2003-2021年总共有: 6550 条路径
6550


7. HasTransaction, ManagerOf, ShareholderOf-1

In [11]:
total_num = cal_3hop_metapath(PATH_NAME='trans_mana_gud(T)')
print(total_num) # 10894

2003 0 (4111, 4111)
2004 4 (4111, 4111)
2005 12 (4111, 4111)
2006 75 (4111, 4111)
2007 33 (4111, 4111)
2008 80 (4111, 4111)
2009 161 (4111, 4111)
2010 148 (4111, 4111)
2011 170 (4111, 4111)
2012 174 (4111, 4111)
2013 256 (4111, 4111)
2014 957 (4111, 4111)
2015 1827 (4111, 4111)
2016 1545 (4111, 4111)
2017 1112 (4111, 4111)
2018 856 (4111, 4111)
2019 921 (4111, 4111)
2020 1162 (4111, 4111)
2021 1401 (4111, 4111)
trans_mana_gud(T) 2003-2021年总共有: 10894 条路径
10894


8. ManagerOf-1, ManagerOf, HasTransaction

In [12]:
total_num = cal_3hop_metapath(PATH_NAME='mana(T)_mana_trans')
print(total_num) # 90563

2003 1012 (4111, 4111)
2004 1034 (4111, 4111)
2005 1062 (4111, 4111)
2006 1332 (4111, 4111)
2007 1311 (4111, 4111)
2008 1816 (4111, 4111)
2009 1865 (4111, 4111)
2010 2654 (4111, 4111)
2011 3253 (4111, 4111)
2012 3627 (4111, 4111)
2013 4226 (4111, 4111)
2014 5309 (4111, 4111)
2015 5926 (4111, 4111)
2016 6809 (4111, 4111)
2017 8262 (4111, 4111)
2018 8365 (4111, 4111)
2019 10122 (4111, 4111)
2020 11190 (4111, 4111)
2021 11388 (4111, 4111)
mana(T)_mana_trans 2003-2021年总共有: 90563 条路径
90563


9. ManagerOf-1, ManagerOf, ShareholderOf

In [13]:
total_num = cal_3hop_metapath(PATH_NAME='mana(T)_mana_gud')
print(total_num) # 40989

2003 1055 (4111, 4111)
2004 1152 (4111, 4111)
2005 1282 (4111, 4111)
2006 1230 (4111, 4111)
2007 1190 (4111, 4111)
2008 1194 (4111, 4111)
2009 1243 (4111, 4111)
2010 1429 (4111, 4111)
2011 1633 (4111, 4111)
2012 1670 (4111, 4111)
2013 1863 (4111, 4111)
2014 1878 (4111, 4111)
2015 2142 (4111, 4111)
2016 2861 (4111, 4111)
2017 3482 (4111, 4111)
2018 3369 (4111, 4111)
2019 3863 (4111, 4111)
2020 4176 (4111, 4111)
2021 4277 (4111, 4111)
mana(T)_mana_gud 2003-2021年总共有: 40989 条路径
40989


10. ManagerOf-1, ManagerOf, ShareholderOf-1

In [14]:
total_num = cal_3hop_metapath(PATH_NAME='mana(T)_mana_gud(T)')
print(total_num) # 226293

2003 1699 (4111, 4111)
2004 2178 (4111, 4111)
2005 2448 (4111, 4111)
2006 2926 (4111, 4111)
2007 3591 (4111, 4111)
2008 3633 (4111, 4111)
2009 5443 (4111, 4111)
2010 7470 (4111, 4111)
2011 8113 (4111, 4111)
2012 8499 (4111, 4111)
2013 9920 (4111, 4111)
2014 18090 (4111, 4111)
2015 24467 (4111, 4111)
2016 24203 (4111, 4111)
2017 22282 (4111, 4111)
2018 17127 (4111, 4111)
2019 17904 (4111, 4111)
2020 20543 (4111, 4111)
2021 25757 (4111, 4111)
mana(T)_mana_gud(T) 2003-2021年总共有: 226293 条路径
226293


#### 不含managerOf 和 kinshipOf: 14。2023.06.02，这14条不要了
1. HasTransaction, HasTransaction, HasTransaction
2. ShareholderOf, HasTransaction, HasTransaction
3. ShareholderOf-1, HasTransaction, HasTransaction
4. HasTransaction, ShareholderOf, HasTransaction
5. ShareholderOf-1, ShareholderOf, HasTransaction
6. ShareholderOf, ShareholderOf-1, HasTransaction
7. ShareholderOf, ShareholderOf, HasTransaction
8. ShareholderOf-1, ShareholderOf-1, HasTransaction
9. ShareholderOf-1, HasTransaction, ShareholderOf
10. ShareholderOf, HasTransaction, ShareholderOf
11. ShareholderOf-1, ShareholderOf, ShareholderOf
12. ShareholderOf, ShareholderOf-1, ShareholderOf
13. ShareholderOf, ShareholderOf, ShareholderOf-1
14. ShareholderOf, ShareholderOf, ShareholderOf


1. HasTransaction, HasTransaction, HasTransaction

In [7]:
total_num = cal_3hop_metapath(PATH_NAME='trans_trans_trans')
print(total_num) # 74830

2003 587 (4111, 4111)
2004 561 (4111, 4111)
2005 656 (4111, 4111)
2006 783 (4111, 4111)
2007 881 (4111, 4111)
2008 1346 (4111, 4111)
2009 1664 (4111, 4111)
2010 1976 (4111, 4111)
2011 2503 (4111, 4111)
2012 2775 (4111, 4111)
2013 3237 (4111, 4111)
2014 3540 (4111, 4111)
2015 3954 (4111, 4111)
2016 4965 (4111, 4111)
2017 6224 (4111, 4111)
2018 6985 (4111, 4111)
2019 9376 (4111, 4111)
2020 10535 (4111, 4111)
2021 12282 (4111, 4111)
trans_trans_trans 2003-2021年总共有: 74830 条路径
74830


2. ShareholderOf, HasTransaction, HasTransaction

In [8]:
total_num = cal_3hop_metapath(PATH_NAME='gud_trans_trans')
print(total_num) # 120197

2003 632 (4111, 4111)
2004 994 (4111, 4111)
2005 1141 (4111, 4111)
2006 1365 (4111, 4111)
2007 1614 (4111, 4111)
2008 1973 (4111, 4111)
2009 2918 (4111, 4111)
2010 3621 (4111, 4111)
2011 4145 (4111, 4111)
2012 4532 (4111, 4111)
2013 4959 (4111, 4111)
2014 7958 (4111, 4111)
2015 11569 (4111, 4111)
2016 11247 (4111, 4111)
2017 10753 (4111, 4111)
2018 9992 (4111, 4111)
2019 11151 (4111, 4111)
2020 12978 (4111, 4111)
2021 16655 (4111, 4111)
gud_trans_trans 2003-2021年总共有: 120197 条路径
120197


3. ShareholderOf-1, HasTransaction, HasTransaction

In [9]:
total_num = cal_3hop_metapath(PATH_NAME='gud(T)_trans_trans')
print(total_num)

2003 618 (4111, 4111)
2004 746 (4111, 4111)
2005 862 (4111, 4111)
2006 1081 (4111, 4111)
2007 992 (4111, 4111)
2008 1360 (4111, 4111)
2009 1509 (4111, 4111)
2010 1689 (4111, 4111)
2011 2263 (4111, 4111)
2012 2551 (4111, 4111)
2013 3011 (4111, 4111)
2014 4207 (4111, 4111)
2015 5421 (4111, 4111)
2016 5041 (4111, 4111)
2017 6474 (4111, 4111)
2018 9683 (4111, 4111)
2019 13923 (4111, 4111)
2020 21854 (4111, 4111)
2021 27087 (4111, 4111)
gud(T)_trans_trans 2003-2021年总共有: 110372 条路径
110372


4. HasTransaction, ShareholderOf, HasTransaction

### 统计各个meta-path的数量:

In [103]:
'''
P1: gud_gud
P2: gu_qin_gu
P3: gud_trans
P4: mana_gud
P5: mana_mana
P6: mana_qin_gud
P7: mana_qin_mana
P8: sub_trans
P9: trans_trans
P10: trans_trans_trans
'''

def path_stats(PATH_NAME):
    N_by_year = []
    for year in range(2003,2022):
        path = mmread(os.path.join('mat_by_year',str(year),f'metapath/{PATH_NAME}.mtx'))
        N_by_year.append(path.nnz)
    return N_by_year, np.sum(N_by_year)

N_by_year, n_agg = path_stats(PATH_NAME='trans_trans_trans')
print(N_by_year, n_agg)


[587, 561, 656, 783, 881, 1346, 1664, 1976, 2503, 2775, 3237, 3540, 3954, 4965, 6224, 6985, 9376, 10535, 12282] 74830


查看 subsidiary 和 shareholder 的实体重复率

In [17]:
len(set(gudong.shareholder.tolist()) & set(subfirm.subsidiary.tolist())), \
    len(set(gudong.shareholder.tolist())), len(set(subfirm.subsidiary.tolist()))

(120278, 184869)

In [ ]:
mat_a = coo_matrix(np.array([
                    [1,0,0,0,1,0,0],
                    [1,1,0,0,0,0,1],
                ]))
mat_b = coo_matrix(np.array([
                    [1,1],
                    [0,0],
                    [0,0],
                    [0,0],
                    [1,0],
                    [0,0],
                    [0,1]
                ]))
mat_c = coo_matrix(np.array([
                    [1,1,1],
                    [0,1,1],
                ]))
mat_d = coo_matrix(np.array([
                    [1,1,0],
                    [0,1,0],
                    [0,0,1]
                ]))
'''coo = cal_metapath(mat_c, mat_d,[0,1,2])
coo_den = coo.todense()
coo_den'''
mat_a_T = trans(mat_a)
mat_a_T = mat_a_T.todense()
a = np.array([
  [1,0,0,0,1,0,0],
  [1,1,0,0,0,0,1],
])
b = np.array([
  [0,0,1,0,1,0,1],
  [1,1,0,1,0,1,0],
])
np.matmul(a, b.T)
np.matmul(b, a.T)

outdegree = np.array([list(mat_b.row).count(i) for i in range(mat_a.shape[1])])
print(outdegree)
outdegree_reverse = np.array([list(mat_a.col).count(i) for i in range(mat_b.shape[0])])
np.sum(1/2 * 1/outdegree[[0,4]])

In [ ]:
import torch
trans_trans_trans = mmread('mat_by_year/2018/metapath/trans_trans.mtx')
torch.from_numpy(trans_trans_trans.todense())
trans_trans_trans.data.shape

In [ ]:
import meta_3rel_v2, meta_3rel_v1, meta_4rel_v2

ab = meta_2rel_v2.cal_metapath(mat_a, mat_b, del_diagnoal=False)
ab = ab.todense()

ac = meta_3rel_v2.cal_metapath(mat_a, mat_b, mat_c, del_diagnoal=False)
ac = ac.todense()
ac

ad = meta_4rel_v2.cal_metapath(mat_a, mat_b, mat_c, mat_d, del_diagnoal=False)
ad = ad.todense()
ad

In [ ]:
92+134+152+198+198+68+218+84+128+36+48

### 简单地穷举元路径

In [102]:
import itertools
def find_permutations(elements, sample_num):
    permutations = list(itertools.product(elements, repeat=sample_num))
    return permutations

# 示例输入
n = 6
elements = ['shareholderOf','hasTransaction','managerOf','subsidiaryOf','kinshipOf']

result_3 = find_permutations(elements, 3)
result_3

[('shareholderOf', 'shareholderOf', 'shareholderOf'),
 ('shareholderOf', 'shareholderOf', 'hasTransaction'),
 ('shareholderOf', 'shareholderOf', 'managerOf'),
 ('shareholderOf', 'shareholderOf', 'subsidiaryOf'),
 ('shareholderOf', 'shareholderOf', 'kinshipOf'),
 ('shareholderOf', 'hasTransaction', 'shareholderOf'),
 ('shareholderOf', 'hasTransaction', 'hasTransaction'),
 ('shareholderOf', 'hasTransaction', 'managerOf'),
 ('shareholderOf', 'hasTransaction', 'subsidiaryOf'),
 ('shareholderOf', 'hasTransaction', 'kinshipOf'),
 ('shareholderOf', 'managerOf', 'shareholderOf'),
 ('shareholderOf', 'managerOf', 'hasTransaction'),
 ('shareholderOf', 'managerOf', 'managerOf'),
 ('shareholderOf', 'managerOf', 'subsidiaryOf'),
 ('shareholderOf', 'managerOf', 'kinshipOf'),
 ('shareholderOf', 'subsidiaryOf', 'shareholderOf'),
 ('shareholderOf', 'subsidiaryOf', 'hasTransaction'),
 ('shareholderOf', 'subsidiaryOf', 'managerOf'),
 ('shareholderOf', 'subsidiaryOf', 'subsidiaryOf'),
 ('shareholderOf', 's

In [32]:
# 路径长度为2
elements = ['shareholderOf','hasTransaction','managerOf','subsidiaryOf']
result_2 = find_permutations(elements, 2)
for path in result_2:
    print(f'(C1, {path[0]}, C3,{path[1]}, C2)')

(C1, shareholderOf, C3,shareholderOf, C2)
(C1, shareholderOf, C3,hasTransaction, C2)
(C1, shareholderOf, C3,managerOf, C2)
(C1, shareholderOf, C3,subsidiaryOf, C2)
(C1, hasTransaction, C3,shareholderOf, C2)
(C1, hasTransaction, C3,hasTransaction, C2)
(C1, hasTransaction, C3,managerOf, C2)
(C1, hasTransaction, C3,subsidiaryOf, C2)
(C1, managerOf, C3,shareholderOf, C2)
(C1, managerOf, C3,hasTransaction, C2)
(C1, managerOf, C3,managerOf, C2)
(C1, managerOf, C3,subsidiaryOf, C2)
(C1, subsidiaryOf, C3,shareholderOf, C2)
(C1, subsidiaryOf, C3,hasTransaction, C2)
(C1, subsidiaryOf, C3,managerOf, C2)
(C1, subsidiaryOf, C3,subsidiaryOf, C2)


In [37]:
len(result_2)

16

### meta-path的schema: 用 heterogeneous random_walk 算法统计所有元路径
在 chatgpt 搜“给定一个关系异构图的邻接矩阵结构 XXX = {YYYY}, 其中节点id为0和1，('managerOf', 0)二元组是关系和尾节点，如何设计一个起始节点为0，终止节点也为0，路径长度为3的random walk算法，要求输出所有路径”


下面这个算法 num_walks 取不一样的值，输出的路径条数不一样。所以和 ppt 里总结的30条结果还不太一样

In [100]:
# 用 dgl
import torch
import random
# 0: company, 1: people
# relation_type = ['shareholderOf','hasTransaction','managerOf','kinshipOf']
business_net = dgl.heterograph({
    ('company', 'shareholderOf', 'company'): ([0,1], [0,0]),
    ('company', 'shareholderOf(T)', 'company & people'): ([0,1], [0,0]),
    ('company', 'hasTransaction', 'company & people'): ([0,0,1], [0,1,0]),
    ('people', 'managerOf', 'company'): ([1], [0]),
    ('people', 'kinshipOf', 'people'): ([1], [1]),
})

adj = {
    0 : [('shareholderOf', 0),
         ('shareholderOf(T)', 0),
         ('shareholderOf(T)', 1), 
         ('managerOf(T)', 1), 
         ('hasTransaction', 0), 
         ('hasTransaction', 1)
        ], 
    1 : [('managerOf', 0),
         ('kinshipOf', 1), 
         ('hasTransaction', 0), 
         ('shareholderOf', 0)]
}

def random_walk_heterogeneous(adj, current_node, end_node, num_steps, path, paths):
    if current_node == end_node and len(path) == num_steps:
        paths.append(list(path))
        return

    if len(path) == num_steps:
        return

    neighbors = adj[current_node]
    
    edge_type, next_node = random.choice(neighbors)
    path.append((edge_type, next_node))
    random_walk_heterogeneous(adj, next_node, end_node, num_steps, path, paths)
    path.pop()


def generate_random_walks(graph, start_node, end_node, num_steps, num_walks):
    walks = []
    for _ in range(num_walks):
        path = [(None, start_node)]  # 路径的起始节点
        paths = []
        random_walk_heterogeneous(graph, start_node, end_node, num_steps, path, paths)
        walks.extend(paths)
    return walks

    
# 构建关系异构图的邻接列表表示


start = 0
end = 0
steps = 4
num_walks = 3000 # walk 多少次，每一次walk一条路，但不一定能到终点

walks = generate_random_walks(adj, start, end, steps, num_walks)


# walks = [' - '.join([str(path[0][1]), path[1][0], str(path[1][1]), path[2][0], str(path[2][1]),path[3][0],str(path[3][1])]) for path in walks]
walks = [' - '.join([str(path[0][1]), path[1][0], path[2][0], path[3][0],str(path[3][1])]) for path in walks]

print(f"所有符合条件的路径共 {len(set(walks))} 条:")
for path in sorted(set(walks)):
    print(path)

所有符合条件的路径共 66 条:
0 - hasTransaction - hasTransaction - hasTransaction - 0
0 - hasTransaction - hasTransaction - managerOf - 0
0 - hasTransaction - hasTransaction - shareholderOf - 0
0 - hasTransaction - hasTransaction - shareholderOf(-1) - 0
0 - hasTransaction - kinshipOf - hasTransaction - 0
0 - hasTransaction - kinshipOf - managerOf - 0
0 - hasTransaction - kinshipOf - shareholderOf - 0
0 - hasTransaction - managerOf - hasTransaction - 0
0 - hasTransaction - managerOf - shareholderOf - 0
0 - hasTransaction - managerOf - shareholderOf(-1) - 0
0 - hasTransaction - managerOf(-1) - hasTransaction - 0
0 - hasTransaction - managerOf(-1) - managerOf - 0
0 - hasTransaction - managerOf(-1) - shareholderOf - 0
0 - hasTransaction - shareholderOf - hasTransaction - 0
0 - hasTransaction - shareholderOf - shareholderOf - 0
0 - hasTransaction - shareholderOf - shareholderOf(-1) - 0
0 - hasTransaction - shareholderOf(-1) - hasTransaction - 0
0 - hasTransaction - shareholderOf(-1) - managerOf - 0
0 -

发现当 num_walks > 1000 时，所有符合条件的路径定格在 66 条，所以总的就是 66 条，但是这 66 条里面还有一些是对称的，可以去掉一半